In [1]:
import pandas as pd
import numpy as np

import sys
import docplex.mp
import cplex

In [2]:
## Teams to schedule game for
teams = ['Michigan State', 'Michigan', 'Ohio State', 'Penn State', 'Rutgers',
         'Iowa', 'Minnesota', 'Nebraska', 'Purdue', 'Wisconsin']

In [3]:
## data frame of teams
teams_df = pd.DataFrame(teams)
teams_df.columns = ["Teams"]
teams_df

,Teams
0,Michigan State
1,Michigan
2,Ohio State
3,Penn State
4,Rutgers
5,Iowa
6,Minnesota
7,Nebraska
8,Purdue
9,Wisconsin


In [4]:
## numbers to identify the teams
team_nums = range(len(teams))
team_nums

range(0, 10)

In [5]:
from collections import namedtuple

## create a namedtuple with each team name
game = namedtuple("game",["team_1","team_2"])

## generate all unique games to play (ie each unique team combination)
unique_games_to_play = {game(t1,t2) for t1 in team_nums for t2 in team_nums if t1 < t2}
unique_games_to_play

{game(team_1=0, team_2=1),
 game(team_1=0, team_2=2),
 game(team_1=0, team_2=3),
 game(team_1=0, team_2=4),
 game(team_1=0, team_2=5),
 game(team_1=0, team_2=6),
 game(team_1=0, team_2=7),
 game(team_1=0, team_2=8),
 game(team_1=0, team_2=9),
 game(team_1=1, team_2=2),
 game(team_1=1, team_2=3),
 game(team_1=1, team_2=4),
 game(team_1=1, team_2=5),
 game(team_1=1, team_2=6),
 game(team_1=1, team_2=7),
 game(team_1=1, team_2=8),
 game(team_1=1, team_2=9),
 game(team_1=2, team_2=3),
 game(team_1=2, team_2=4),
 game(team_1=2, team_2=5),
 game(team_1=2, team_2=6),
 game(team_1=2, team_2=7),
 game(team_1=2, team_2=8),
 game(team_1=2, team_2=9),
 game(team_1=3, team_2=4),
 game(team_1=3, team_2=5),
 game(team_1=3, team_2=6),
 game(team_1=3, team_2=7),
 game(team_1=3, team_2=8),
 game(team_1=3, team_2=9),
 game(team_1=4, team_2=5),
 game(team_1=4, team_2=6),
 game(team_1=4, team_2=7),
 game(team_1=4, team_2=8),
 game(team_1=4, team_2=9),
 game(team_1=5, team_2=6),
 game(team_1=5, team_2=7),
 

In [6]:
## generate all games (each team plays twice)
all_games_to_play = {game : 2 for game in unique_games_to_play}

all_games_to_play

{game(team_1=5, team_2=9): 2,
 game(team_1=4, team_2=7): 2,
 game(team_1=1, team_2=3): 2,
 game(team_1=6, team_2=9): 2,
 game(team_1=4, team_2=8): 2,
 game(team_1=5, team_2=6): 2,
 game(team_1=0, team_2=2): 2,
 game(team_1=2, team_2=8): 2,
 game(team_1=0, team_2=7): 2,
 game(team_1=8, team_2=9): 2,
 game(team_1=1, team_2=6): 2,
 game(team_1=3, team_2=7): 2,
 game(team_1=2, team_2=5): 2,
 game(team_1=0, team_2=3): 2,
 game(team_1=5, team_2=8): 2,
 game(team_1=1, team_2=2): 2,
 game(team_1=4, team_2=9): 2,
 game(team_1=6, team_2=7): 2,
 game(team_1=2, team_2=9): 2,
 game(team_1=1, team_2=5): 2,
 game(team_1=3, team_2=6): 2,
 game(team_1=0, team_2=4): 2,
 game(team_1=2, team_2=6): 2,
 game(team_1=4, team_2=5): 2,
 game(team_1=1, team_2=4): 2,
 game(team_1=3, team_2=9): 2,
 game(team_1=0, team_2=5): 2,
 game(team_1=1, team_2=9): 2,
 game(team_1=2, team_2=3): 2,
 game(team_1=0, team_2=8): 2,
 game(team_1=3, team_2=5): 2,
 game(team_1=0, team_2=1): 2,
 game(team_1=2, team_2=7): 2,
 game(team

In [7]:
## create model environment
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Darwin 64bit
* Python is present, version is 3.6.8
* docplex is present, version is (2, 9, 141)
* CPLEX wrapper is present, version is 12.8.0.0, located at: /anaconda3/lib/python3.6/site-packages


In [8]:
from docplex.mp.model import Model

## create sport scheduling mp model
mdl = Model("sport_scheduling")

In [9]:
## week numbers
weeks = range(1, 19)

In [10]:
## add possibility for each game to be played on any given week. This holds all possibilities
game_week_possibilities = mdl.binary_var_matrix(all_games_to_play, weeks, lambda ij: "x_%s_%d" %(str(ij[0]), ij[1]))
game_week_possibilities

{(game(team_1=5, team_2=9),
  1): docplex.mp.Var(type=B,name='x_game(team_1=5, team_2=9)_1'),
 (game(team_1=5, team_2=9),
  2): docplex.mp.Var(type=B,name='x_game(team_1=5, team_2=9)_2'),
 (game(team_1=5, team_2=9),
  3): docplex.mp.Var(type=B,name='x_game(team_1=5, team_2=9)_3'),
 (game(team_1=5, team_2=9),
  4): docplex.mp.Var(type=B,name='x_game(team_1=5, team_2=9)_4'),
 (game(team_1=5, team_2=9),
  5): docplex.mp.Var(type=B,name='x_game(team_1=5, team_2=9)_5'),
 (game(team_1=5, team_2=9),
  6): docplex.mp.Var(type=B,name='x_game(team_1=5, team_2=9)_6'),
 (game(team_1=5, team_2=9),
  7): docplex.mp.Var(type=B,name='x_game(team_1=5, team_2=9)_7'),
 (game(team_1=5, team_2=9),
  8): docplex.mp.Var(type=B,name='x_game(team_1=5, team_2=9)_8'),
 (game(team_1=5, team_2=9),
  9): docplex.mp.Var(type=B,name='x_game(team_1=5, team_2=9)_9'),
 (game(team_1=5, team_2=9),
  10): docplex.mp.Var(type=B,name='x_game(team_1=5, team_2=9)_10'),
 (game(team_1=5, team_2=9),
  11): docplex.mp.Var(type=B,n

In [11]:
## add constraint that every game must be played
mdl.add_constraints(mdl.sum(game_week_possibilities[game, week]  for week in weeks) == all_games_to_play[game]
                   for game in unique_games_to_play)
mdl.print_information()

Model: sport_scheduling
 - number of variables: 810
   - binary=810, integer=0, continuous=0
 - number of constraints: 45
   - linear=45
 - parameters: defaults


In [12]:
## add constraint that every team must play exactly once a week
mdl.add_constraints(mdl.sum(game_week_possibilities[game, week] 
                            for game in unique_games_to_play 
                            if (game.team_1 == team_num or game.team_2 == team_num)) == 1
                    for week in weeks for team_num in team_nums)
mdl.print_information()

Model: sport_scheduling
 - number of variables: 810
   - binary=810, integer=0, continuous=0
 - number of constraints: 225
   - linear=225
 - parameters: defaults


In [13]:
mdl.print_information()

## solve the model
assert mdl.solve(), "!!! Solve of the model fails"
mdl.report()

Model: sport_scheduling
 - number of variables: 810
   - binary=810, integer=0, continuous=0
 - number of constraints: 225
   - linear=225
 - parameters: defaults
* model sport_scheduling solved with objective = 0


In [14]:
## dictionary for linking team number and team name
team_league = dict(list({t : teams[t] for t in range(10) }.items()))
team_league

{0: 'Michigan State',
 1: 'Michigan',
 2: 'Ohio State',
 3: 'Penn State',
 4: 'Rutgers',
 5: 'Iowa',
 6: 'Minnesota',
 7: 'Nebraska',
 8: 'Purdue',
 9: 'Wisconsin'}

In [15]:
## solution named tuple
schedule = namedtuple("schedule",["week", "team_1", "team_2"])

game_schedule = [schedule(week, team_league[game.team_1], team_league[game.team_2]) 
                 for game in unique_games_to_play 
                 for week in weeks 
                 if game_week_possibilities[game, week].solution_value == 1]
game_schedule

[schedule(week=3, team_1='Iowa', team_2='Wisconsin'),
 schedule(week=8, team_1='Iowa', team_2='Wisconsin'),
 schedule(week=1, team_1='Rutgers', team_2='Nebraska'),
 schedule(week=7, team_1='Rutgers', team_2='Nebraska'),
 schedule(week=4, team_1='Michigan', team_2='Penn State'),
 schedule(week=10, team_1='Michigan', team_2='Penn State'),
 schedule(week=10, team_1='Minnesota', team_2='Wisconsin'),
 schedule(week=14, team_1='Minnesota', team_2='Wisconsin'),
 schedule(week=3, team_1='Rutgers', team_2='Purdue'),
 schedule(week=13, team_1='Rutgers', team_2='Purdue'),
 schedule(week=16, team_1='Iowa', team_2='Minnesota'),
 schedule(week=18, team_1='Iowa', team_2='Minnesota'),
 schedule(week=11, team_1='Michigan State', team_2='Ohio State'),
 schedule(week=16, team_1='Michigan State', team_2='Ohio State'),
 schedule(week=2, team_1='Ohio State', team_2='Purdue'),
 schedule(week=17, team_1='Ohio State', team_2='Purdue'),
 schedule(week=3, team_1='Michigan State', team_2='Nebraska'),
 schedule(we